# VLM Benchmark for Object Property Abstraction

This notebook implements a benchmark for evaluating Vision Language Models (VLMs) on object property abstraction and visual question answering (VQA) tasks. The benchmark includes three types of questions:

1. Direct Recognition
2. Property Inference
3. Counterfactual Reasoning

And three types of images:
- REAL
- ANIMATED
- AI GENERATED

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [1]:
# Install required packages
# %pip install transformers torch Pillow tqdm bitsandbytes accelerate

In [2]:
%pip install flash-attn #--no-build-isolation 

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import required libraries
import torch
import json
from pathlib import Path
from PIL import Image
import gc
import re
from tqdm import tqdm
from typing import Dict

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
import numpy as np
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode

/var/scratch/ave303/anaconda3/envs/op_bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Benchmark Tester Class

This class handles the evaluation of models against our benchmark.

In [5]:
class BenchmarkTester:
    def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        with open(benchmark_path, 'r') as f:
            self.benchmark = json.load(f)
        self.data_dir = data_dir

    def clean_answer(self, answer):
        """Extract number and reasoning from the model's answer."""
        # Try to extract number and reasoning using regex
        import re
        pattern = r'(\d+)\s*\[(.*?)\]'
        match = re.search(pattern, answer)
        
        if match:
            number = match.group(1)
            objects = [obj.strip() for obj in match.group(2).split(',')]
            return {
                "count": number,
                "reasoning": objects
            }
        else:
            # Fallback if format isn't matched
            numbers = re.findall(r'\d+', answer)
            return {
                "count": numbers[0] if numbers else "0",
                "reasoning": []
            }

    # IMAGENET_MEAN = (0.485, 0.456, 0.406)
    # IMAGENET_STD = (0.229, 0.224, 0.225)

    IMAGENET_MEAN = (0.5, 0.5, 0.5)
    IMAGENET_STD = (0.5, 0.5, 0.5)


    def build_transform(self, input_size):
        MEAN, STD = self.IMAGENET_MEAN, self.IMAGENET_STD
        transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])
        return transform

    def find_closest_aspect_ratio(self, aspect_ratio, target_ratios, width, height, image_size):
        best_ratio_diff = float('inf')
        best_ratio = (1, 1)
        area = width * height
        for ratio in target_ratios:
            target_aspect_ratio = ratio[0] / ratio[1]
            ratio_diff = abs(aspect_ratio - target_aspect_ratio)
            if ratio_diff < best_ratio_diff:
                best_ratio_diff = ratio_diff
                best_ratio = ratio
            elif ratio_diff == best_ratio_diff:
                if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                    best_ratio = ratio
        return best_ratio

    def dynamic_preprocess(self, image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
        orig_width, orig_height = image.size
        aspect_ratio = orig_width / orig_height
    
        # calculate the existing image aspect ratio
        target_ratios = set(
            (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
            i * j <= max_num and i * j >= min_num)
        target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    
        # find the closest aspect ratio to the target
        target_aspect_ratio = self.find_closest_aspect_ratio(
            aspect_ratio, target_ratios, orig_width, orig_height, image_size)

        # calculate the target width and height
        target_width = image_size * target_aspect_ratio[0]
        target_height = image_size * target_aspect_ratio[1]
        blocks = target_aspect_ratio[0] * target_aspect_ratio[1]
    
        # resize the image
        resized_img = image.resize((target_width, target_height))
        processed_images = []
        for i in range(blocks):
            box = (
                (i % (target_width // image_size)) * image_size,
                (i // (target_width // image_size)) * image_size,
                ((i % (target_width // image_size)) + 1) * image_size,
                ((i // (target_width // image_size)) + 1) * image_size
            )
            # split the image
            split_img = resized_img.crop(box)
            processed_images.append(split_img)
        assert len(processed_images) == blocks
        if use_thumbnail and len(processed_images) != 1:
            thumbnail_img = image.resize((image_size, image_size))
            processed_images.append(thumbnail_img)
        return processed_images

    def load_image(self, image_file, input_size=448, max_num=12):     #internvl -> 448, 12 || ristretto -> 384, 10
        image = Image.open(image_file).convert('RGB')
        transform = self.build_transform(input_size=input_size)
        images = self.dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(image) for image in images]
        pixel_values = torch.stack(pixel_values)
        return pixel_values
    
    def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
        results = []
        print(f"\nEvaluating {model_name}...")
        print(f"Using device: {self.device}")
        
        # Force garbage collection before starting
        gc.collect()
        torch.cuda.empty_cache()

        try:
            images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
            total_images = len(images)
            
            for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
                try:
                    print(f"\nProcessing image {idx+1}/{total_images}: {image_data['image_id']}")
                    image_path = Path(self.data_dir)/image_data['path']
                    if not image_path.exists():
                        print(f"Warning: Image not found at {image_path}")
                        continue
                    
                    # Load and preprocess image
                    # image = Image.open(image_path).convert("RGB")
                    image_results = []  # Store results for current image
                    
                    for question in image_data['questions']:
                        try:
                            print(f"Question: {question['question']}")
                            
                            # Clear cache before processing each question
                            torch.cuda.empty_cache()

                            # set the max number of tiles in `max_num`
                            
                            pixel_values = self.load_image(image_path, max_num=12).to(torch.float16).cuda()
                            generation_config = dict(max_new_tokens=1024, do_sample=True)
                            # prompt = f'<image>\n {question["question"]} Provide the total count of the objects and then list the objects, separated by commas. \n Format: <number> [<object1>, <object2>, <object3>, ...]'
                            # Answer with the total number(numerical) followed by the objects within square brackets' #Answer format: total number(numerical) objects(within square brackets)'
                            prompt = f'<image>\n {question["question"]} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]'
                            # prompt = f'<image>\n {question["question"]} Answer format: total count  [list of objects]'
                            answer = model.chat(processor, pixel_values, prompt, generation_config)
                            
                            cleaned_answer = self.clean_answer(answer)
                            
                            image_results.append({
                                "image_id": image_data["image_id"],
                                "image_type": image_data["image_type"],
                                "question_id": question["id"],
                                "question": question["question"],
                                "ground_truth": question["answer"],
                                "model_answer": cleaned_answer["count"],
                                "model_reasoning": cleaned_answer["reasoning"],
                                "raw_answer": answer,  # Keep raw answer for debugging
                                "property_category": question["property_category"]
                            })
                            
                            # Clear memory
                            # del outputs, inputs
                            torch.cuda.empty_cache()
                            
                        except Exception as e:
                            print(f"Error processing question: {str(e)}")
                            continue
                    
                    # Add results from this image
                    results.extend(image_results)
                    
                    # Save intermediate results only every 2 images or if it's the last image
                    if (idx + 1) % 2 == 0 or idx == total_images - 1:
                        with open(f"{save_path}_checkpoint.json", 'w') as f:
                            json.dump(results, f, indent=4)
                            
                except Exception as e:
                    print(f"Error processing image {image_data['image_id']}: {str(e)}")
                    continue
            
            # Save final results
            if results:
                with open(save_path, 'w') as f:
                    json.dump(results, f, indent=4)
            
        except Exception as e:
            print(f"An error occurred during evaluation: {str(e)}")
            if results:
                with open(f"{save_path}_error_state.json", 'w') as f:
                    json.dump(results, f, indent=4)
        
        return results

## Test InternVL2.5
Let's evaluate the InternVL2_5-4B-MPO model

In [6]:
# def split_model(model_name):
#     import math
#     device_map = {}
#     world_size = torch.cuda.device_count()
#     config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
#     num_layers = config.llm_config.num_hidden_layers
#     # num_layers = {
#     #     'InternVL2_5-1B': 24, 'InternVL2_5-2B': 24, 'InternVL2_5-4B': 36, 'InternVL2_5-8B': 32,
#     #     'InternVL2_5-26B': 48, 'InternVL2_5-38B': 64, 'InternVL2_5-78B': 80}[model_name]
#     # Since the first GPU will be used for ViT, treat it as half a GPU.
#     num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
#     num_layers_per_gpu = [num_layers_per_gpu] * world_size
#     num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
#     layer_cnt = 0
#     for i, num_layer in enumerate(num_layers_per_gpu):
#         for j in range(num_layer):
#             device_map[f'language_model.model.layers.{layer_cnt}'] = i
#             layer_cnt += 1
#     device_map['vision_model'] = 0
#     device_map['mlp1'] = 0
#     device_map['language_model.model.tok_embeddings'] = 0
#     device_map['language_model.model.embed_tokens'] = 0
#     device_map['language_model.output'] = 0
#     device_map['language_model.model.norm'] = 0
#     device_map['language_model.model.rotary_emb'] = 0
#     device_map['language_model.lm_head'] = 0
#     device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

#     return device_map

In [7]:
def test_InternVL2_5():
    import torch
    from transformers import AutoTokenizer, AutoModel

    # device_map = split_model('InternVL2_5-4B')
    
    model = AutoModel.from_pretrained(
        "/var/scratch/ave303/models/internvl2.5-8b",
        torch_dtype=torch.float16,
        # load_in_8bit=False,
        low_cpu_mem_usage=True,
        use_flash_attn=True,
        trust_remote_code=True).to('cuda').eval()
    
    tokenizer = AutoTokenizer.from_pretrained("/var/scratch/ave303/models/internvl2.5-8b", trust_remote_code=True, use_fast=False)

    ## InternVL2.5-4B --> performs decently well. slight post processing required
    
    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True

    tester = BenchmarkTester()
    InternVL2_5_results = tester.evaluate_model(
        "InternVL2.5",
        model, 
        tokenizer, 
        "InternVL2.5_results_8bMPO.json", 
        batch_size=50
    )

    # Clean up
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

In [8]:
def test_InternVL3():
    import torch
    from transformers import AutoTokenizer, AutoModel

    # device_map = split_model('InternVL3-8B')
    
    model = AutoModel.from_pretrained(
        "/var/scratch/ave303/models/internvl3-14b",
        torch_dtype=torch.float16,
        load_in_8bit=False,
        low_cpu_mem_usage=True,
        use_flash_attn=True,
        # device_map=device_map,
        trust_remote_code=True).to('cuda').eval()
    
    tokenizer = AutoTokenizer.from_pretrained("/var/scratch/ave303/models/internvl3-14b", trust_remote_code=True, use_fast=False)

    ## InternVL2.5-4B --> performs decently well. slight post processing required
    
    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True

    tester = BenchmarkTester()
    InternVL3_results = tester.evaluate_model(
        "InternVL3",
        model, 
        tokenizer, 
        "InternVL3_results_14b.json", 
        batch_size=50
    )

    # Clean up
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

In [9]:
# def test_Ristretto():
#     import torch
#     from transformers import AutoTokenizer, AutoModel

#     # device_map = split_model('InternVL3-8B')
    
#     model = AutoModel.from_pretrained(
#         "/var/scratch/ave303/models/ristretto-3b",
#         torch_dtype=torch.float16,
#         low_cpu_mem_usage=True,
#         # device_map=device_map,
#         trust_remote_code=True).to('cuda').eval()
    
#     tokenizer = AutoTokenizer.from_pretrained("/var/scratch/ave303/models/ristretto-3b", trust_remote_code=True, use_fast=False)

#     ## InternVL2.5-4B --> performs decently well. slight post processing required
    
#     # Optional: Enable memory efficient attention
#     if hasattr(model.config, 'use_memory_efficient_attention'):
#         model.config.use_memory_efficient_attention = True

#     tester = BenchmarkTester()
#     Ristretto_results = tester.evaluate_model(
#         "Ristretto-3b",
#         model, 
#         tokenizer, 
#         "Ristretto_3b_results.json", 
#         batch_size=25
#     )

#     # Clean up
#     del model, tokenizer
#     torch.cuda.empty_cache()
#     gc.collect()

## Run Evaluation

Now we can run our evaluation. Let's start with the InternVL2.5 model:

In [10]:
# test_InternVL2_5() #8.59 #9.07

In [11]:
test_InternVL3()

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Loading checkpoint shards:   8%|▊         | 1/13 [00:03<00:39,  3.31s/it]

Loading checkpoint shards:  15%|█▌        | 2/13 [00:06<00:32,  2.95s/it]

Loading checkpoint shards:  23%|██▎       | 3/13 [00:08<00:28,  2.85s/it]

Loading checkpoint shards:  31%|███       | 4/13 [00:11<00:24,  2.71s/it]

...

Question: How many objects present are breakable?


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Question: If the bowls with the tomatoes and the chickpeas were emptied into the steaming pot, how many containers would still have something remaining in them?


Processing images: 100%|██████████| 50/50 [02:30<00:00,  3.00s/it]

Processing images: 100%|██████████| 50/50 [02:30<00:00,  3.01s/it]

In [12]:
# test_Ristretto()